# WEIGTHS AND DIMENTIONS MODEL

In [92]:
# Librerias a usar
import shutil
from datetime import datetime
import pyodbc
import pandas as pd
from sqlalchemy import create_engine, inspect, NVARCHAR, Float, Date, text
from sqlalchemy import text
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from pmdarima import auto_arima
from tqdm import tqdm 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import sqlalchemy
from sqlalchemy import create_engine, types
import pyodbc
import smtplib
from email.mime.text import MIMEText
from sqlalchemy import create_engine, Table, Column, MetaData, Integer, String, Float
import os
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [93]:
def send_email_inicio():
    # Configuración del correo
    sender_email = "ecalderon@maxwarehouse.com"  
    sender_password = "Maxwarehouse2025$"  

    # 🔹 Lista de destinatarios
    recipient_list = ["ecalderon@maxwarehouse.com", "vpverbena@maxwarehouse.com"]
    
    # Contenido del correo
    subject = "🔔 Alerta: Inicio de generación de predicciones de pesos"
    body = "El modelo de predicción ha empezado a correr."

    # Crear el mensaje
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = ", ".join(recipient_list)

    try:
        # Conectar al servidor SMTP (ejemplo para Gmail)
        server = smtplib.SMTP("smtp.office365.com", 587)  # Usa "smtp.office365.com" para Outlook
        server.starttls()  # Seguridad TLS
        server.login(sender_email, sender_password)  # Autenticación
        server.sendmail(sender_email, recipient_list, msg.as_string())  # Enviar
        server.quit()
        print("✅ Correo enviado correctamente")
    except Exception as e:
        print(f"⚠ Error al enviar el correo: {e}")

# Llamar a la función al final del proceso
send_email_inicio()

✅ Correo enviado correctamente


## Incorporar reporte semanal de endicia

In [4]:
# --- Configuración ---

# Carpeta donde están los archivos Excel
carpeta = r'C:\Users\ecalderon\OneDrive - BITS - Max Warehouse\Shipping\Weights Process\Reporte Semanal Endicia'
carpeta_destino = r'C:\Users\ecalderon\OneDrive - BITS - Max Warehouse\Shipping\Weights Process\Reporte Semanal Endicia\Procesados'

# Listar todos los archivos .xlsx de la carpeta
excel_files = [f for f in os.listdir(carpeta) if f.endswith('.xlsx')]

# Datos de conexión a SQL Server
server = '10.12.200.59' 
database = 'Maxwarehouse'
table_name = 'endicia_semanal'

# sqlalchemy + pyodbc
connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
engine = create_engine(connection_string)

# Verificar si hay archivos para procesar
if excel_files:
    for excel_file in excel_files:
        try:
            # Ruta completa al archivo
            ruta_archivo = os.path.join(carpeta, excel_file)

            # Leer el archivo Excel
            df = pd.read_excel(ruta_archivo)
            print(f"Archivo leído: {excel_file}")

            # Cargar los datos a SQL Server
            with engine.connect() as connection:
                df.to_sql(table_name, con=connection, if_exists='append', index=False)

            print(f"Datos de {excel_file} cargados exitosamente a SQL Server 🎯")

            # --- Mover y renombrar el archivo ---
            # Obtener la fecha de hoy
            fecha_hoy = datetime.today().strftime('%Y-%m-%d')

            # Armar el nuevo nombre
            nombre_base, extension = os.path.splitext(excel_file)
            nuevo_nombre = f'{nombre_base}_{fecha_hoy}{extension}'

            # Ruta destino con el nuevo nombre
            ruta_destino = os.path.join(carpeta_destino, nuevo_nombre)

            # Mover y renombrar
            shutil.move(ruta_archivo, ruta_destino)
            print(f"Archivo {excel_file} movido y renombrado a: {ruta_destino}")

        except Exception as e:
            print(f"⚠️ Error procesando {excel_file}: {e}")

else:
    print("No se encontraron archivos Excel (.xlsx) en la carpeta. Continuando con el resto del código.")


Archivo leído: Account Transactional Activity.xlsx
Datos de Account Transactional Activity.xlsx cargados exitosamente a SQL Server 🎯
Archivo Account Transactional Activity.xlsx movido y renombrado a: C:\Users\ecalderon\OneDrive - BITS - Max Warehouse\Shipping\Weights Process\Reporte Semanal Endicia\Procesados\Account Transactional Activity_2025-05-06.xlsx


## QUERY PESOS Y DIMENSIONES

In [5]:
server = '10.12.200.59' #ip del DB es 10.11.100.103
database = 'Maxwarehouse'

# Crear el engine usando la cadena de conexión para SQL Server con pyodbc
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Ejecutar consulta 

query = """

SELECT TrackingNumber, ParentSku, SalesStatusEVP, TotalSales, TotalCost FROM [DWH-MaxWarehouse].[Fact].[Sales]
WHERE SalesOrderDateTime >= DATEADD(MONTH, -4, CAST(GETDATE() AS DATE)) 

"""

df_ventas = pd.read_sql(query, con=engine)


In [6]:
server = '10.12.200.59' #ip del DB es 10.11.100.103
database = 'Maxwarehouse'

# Crear el engine usando la cadena de conexión para SQL Server con pyodbc
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Ejecutar consulta 

query = """

WITH BOXFECHAS AS (
    SELECT 
        TRACKING_NUMBER,
        OrderDate,
        WEIGHT AS WeightBox,
        CASE WHEN LEN(DIMENSIONS) >= 5 THEN CAST(SUBSTRING(DIMENSIONS, 1, CHARINDEX('x', DIMENSIONS) - 1) AS FLOAT) ELSE 0.00 END AS DimHeightBOX,
        CASE WHEN LEN(DIMENSIONS) >= 5 THEN CAST(SUBSTRING(DIMENSIONS, CHARINDEX('x', DIMENSIONS) + 1, CHARINDEX('x', DIMENSIONS, CHARINDEX('x', DIMENSIONS) + 1) - CHARINDEX('x', DIMENSIONS) - 1) AS FLOAT) ELSE 0.00 END AS DimLengthBOX,
        CASE WHEN LEN(DIMENSIONS) >= 5 THEN CAST(RIGHT(DIMENSIONS, LEN(DIMENSIONS) - CHARINDEX('x', DIMENSIONS, CHARINDEX('x', DIMENSIONS) + 1)) AS FLOAT) ELSE 0.00 END AS DimWidthBOX,
        CCN_ORDER_NUMBER
    FROM BOX
    WHERE ISDATE(OrderDate) = 1
),

endicia_semana_mes AS (
    SELECT DISTINCT  
        [Postmark_Date] AS TranDate, 
        [Transaction_Date (GMT)] AS InvDate, 
        REPLACE([Tracking Number], '''', '') AS trackingnum, 
        [Weight_(lb)], 
        [Package Height], 
        [Package Length], 
        [Package Width]
    FROM Endiciatransacprovision
    WHERE ISDATE([Transaction_Date (GMT)]) = 1
      AND CAST([Transaction_Date (GMT)] AS date) >= DATEADD(MONTH, -4, GETDATE())
    
    UNION ALL
    
    SELECT DISTINCT  
        [Postmark Date] AS TranDate, 
        [Transaction Date (GMT)] AS InvDate, 
        REPLACE([Tracking Number], '''', '') AS trackingnum, 
        [Weight (lb)], 
        [Package Height], 
        [Package Length], 
        [Package Width]
    FROM endicia_semanal
    WHERE ISDATE([Transaction Date (GMT)]) = 1
      AND CAST([Transaction Date (GMT)] AS date) >= DATEADD(MONTH, -2, GETDATE())
),

Factura3meses AS (

    -- FEDEX
    SELECT DISTINCT
        CAST(InvoiceDate AS date) AS TranDate,
        CAST(InvoiceDate AS date) AS InvDate,
        GroundTrackingIDPrefix + ExpressorGroundTrackingID AS trackingnum,
        MAX(ActualWeightAmount) AS max_weight, 
        COALESCE(NULLIF(DimHeight, ''), CAST(DimHeightBOX AS VARCHAR)) AS DimHeight,
        COALESCE(NULLIF(DimLength, ''), CAST(DimLengthBOX AS VARCHAR)) AS DimLength,
        COALESCE(NULLIF(DimWidth, ''), CAST(DimWidthBOX AS VARCHAR)) AS DimWidth,
        MAX(CCN_ORDER_NUMBER) AS CCN_ORDER_NUMBER
    FROM 
        FEDEXHISTORICO
    LEFT JOIN 
        BOXFECHAS ON GroundTrackingIDPrefix + ExpressorGroundTrackingID = TRACKING_NUMBER
    WHERE 
        CAST(InvoiceDate AS date) >= DATEADD(MONTH, -3, GETDATE())
    GROUP BY 
        CAST(InvoiceDate AS date),
        CAST(InvoiceDate AS date),
        GroundTrackingIDPrefix + ExpressorGroundTrackingID,
        COALESCE(NULLIF(DimHeight, ''), CAST(DimHeightBOX AS VARCHAR)),
        COALESCE(NULLIF(DimLength, ''), CAST(DimLengthBOX AS VARCHAR)),
        COALESCE(NULLIF(DimWidth, ''), CAST(DimWidthBOX AS VARCHAR))

    UNION ALL

    -- UPS (CON PRIORIDAD PackageDimensions)
    SELECT 
    TranDate,
    InvDate,
    TrackingNumber,
    max_weight,
    Valor1 AS DimHeight,
    Valor2 AS DimLength,
    Valor3 AS DimWidth,
    CCN_ORDER_NUMBER
	FROM (
    SELECT 
        CAST(u.TransactionDate AS date) AS TranDate,
        CAST(u.InvoiceDate AS date) AS InvDate,
        u.TrackingNumber,
        CASE 
            WHEN u.EnteredWeight != 0 THEN u.EnteredWeight
            WHEN u.BilledWeight IS NOT NULL AND u.BilledWeight != 0 THEN u.BilledWeight
            ELSE b.WeightBox
        END AS max_weight,
        CASE 
            WHEN COALESCE(u.PackageDimensions, '') <> '' THEN CAST(SUBSTRING(u.PackageDimensions, 1, CHARINDEX('x', u.PackageDimensions) - 1) AS FLOAT)
            WHEN LEN(u.DetailKeyedDim) >= 5 THEN CAST(SUBSTRING(u.DetailKeyedDim, 1, CHARINDEX('x', u.DetailKeyedDim) - 1) AS FLOAT)
            ELSE b.DimHeightBOX
        END AS Valor1,
        CASE 
            WHEN COALESCE(u.PackageDimensions, '') <> '' THEN CAST(SUBSTRING(u.PackageDimensions, CHARINDEX('x', u.PackageDimensions) + 1, CHARINDEX('x', u.PackageDimensions, CHARINDEX('x', u.PackageDimensions) + 1) - CHARINDEX('x', u.PackageDimensions) - 1) AS FLOAT)
            WHEN LEN(u.DetailKeyedDim) >= 5 THEN CAST(SUBSTRING(u.DetailKeyedDim, CHARINDEX('x', u.DetailKeyedDim) + 1, CHARINDEX('x', u.DetailKeyedDim, CHARINDEX('x', u.DetailKeyedDim) + 1) - CHARINDEX('x', u.DetailKeyedDim) - 1) AS FLOAT)
            ELSE b.DimLengthBOX
        END AS Valor2,
        CASE 
            WHEN COALESCE(u.PackageDimensions, '') <> '' THEN CAST(RIGHT(u.PackageDimensions, LEN(u.PackageDimensions) - CHARINDEX('x', u.PackageDimensions, CHARINDEX('x', u.PackageDimensions) + 1)) AS FLOAT)
            WHEN LEN(u.DetailKeyedDim) >= 5 THEN CAST(RIGHT(u.DetailKeyedDim, LEN(u.DetailKeyedDim) - CHARINDEX('x', u.DetailKeyedDim, CHARINDEX('x', u.DetailKeyedDim) + 1)) AS FLOAT)
            ELSE b.DimWidthBOX
        END AS Valor3,
        CCN_ORDER_NUMBER,
        ROW_NUMBER() OVER (
            PARTITION BY u.TrackingNumber
            ORDER BY 
                CASE 
                    WHEN COALESCE(u.PackageDimensions, '') <> '' THEN 1
                    WHEN LEN(u.DetailKeyedDim) >= 5 THEN 2
                    ELSE 3
                END ASC,
                u.InvoiceDate DESC
        ) AS RowNum
    FROM 
        UPSNEWFORMAT u
    LEFT JOIN 
        BOXFECHAS b ON u.TrackingNumber = b.TRACKING_NUMBER
    WHERE 
        CAST(u.InvoiceDate AS date) >= DATEADD(MONTH, -3, GETDATE())
	) AS UPSFiltrado
	WHERE RowNum = 1


    UNION ALL

    -- USPS (Endicia)
    SELECT DISTINCT 
        TranDate,
        InvDate,
        trackingnum,
        COALESCE([Weight_(lb)], b.WeightBox) AS max_weight,
        COALESCE([Package Height], b.DimHeightBOX) AS DimHeight,
        COALESCE([Package Length], b.DimLengthBOX) AS DimLength,
        COALESCE([Package Width], b.DimWidthBOX) AS DimWidth,
        CCN_ORDER_NUMBER
    FROM 
        endicia_semana_mes e
    LEFT JOIN 
        BOXFECHAS b ON b.TRACKING_NUMBER = e.trackingnum
    WHERE 
        trackingnum IS NOT NULL
        AND CAST(InvDate AS date) >= DATEADD(MONTH, -4, GETDATE()) -- USPS 4 meses porque vienen mensual
),

DWVentasUnicas as (
	Select 
		SalesOrderNumber, Sku
	from DWShippingInfo as dws
	where cast(SalesOrderDate as date ) between dateadd(MONTH,-4, cast(GETDATE() as date)) and cast(GETDATE() as date)
	group by SalesOrderNumber, Sku
),

CantidadSkusxVenta as (
	select
		SalesOrderNumber, count(Sku) CantidadSkus
	from DWVentasUnicas dw 
	group by SalesOrderNumber
	having count(Sku) =1
),

PesosfacturaVentas as (
	--Lista los trackings con su peso y le coloca la moda y las frecuencia con que ese peso fue facturado
	select  
		TranDate, InvDate, dw.trackingnum, max_weight, DimHeight, DimLength, DimWidth, mw.Sku, mw.UomCode, mw.UomQuantity
	FROM Factura3meses dw
	INNER JOIN DWShippingInfo  mw
		ON dw.trackingnum = mw.trackingnum
	WHERE dw.trackingnum !=''
	and SalesOrderNumber in (select SalesOrderNumber from CantidadSkusxVenta)
	and cast(UomQuantity as float)/cast(Quantity as float)= 1
),

RankedData AS (
    SELECT 
        TranDate, 
        InvDate, 
        trackingnum, 
        Sku, 
        UomCode, 
        UomQuantity,
        max_weight, 
        DimHeight, 
        DimLength, 
        DimWidth,
        RANK() OVER (
            PARTITION BY TranDate, trackingnum, Sku, UomCode 
            ORDER BY InvDate DESC
        ) AS Ranking
    FROM PesosfacturaVentas
),

Max_Rank as (SELECT 
    TranDate, 
    InvDate, 
    trackingnum, 
    Sku, 
    UomCode, 
    UomQuantity,
    MAX(max_weight) AS Weight, 
    MAX(DimHeight) AS Height, 
    MAX(DimLength) AS Length, 
    MAX(DimWidth) AS Width,
    Ranking
FROM RankedData
GROUP BY TranDate, InvDate, trackingnum, Sku, UomCode, UomQuantity, Ranking
),

EJD_DIMS as (SELECT aa.*, 
       bb.[EJD Weight], 
       bb.[EJD Height], 
       bb.[EJD Length], 
       bb.[EJD Width],
       bb.[BaseUnit]
FROM Max_Rank aa
LEFT JOIN (
    SELECT DISTINCT [EVP SKU], [EVP UOM CODE], [EVP UOM QTY], [EJD Weight], [EJD Height], [EJD Length], [EJD Width], [BaseUnit]
    FROM [stg-MaxWarehouse].[EvpCsv].[EVPAlllSKUsDimNWeight]
) bb
ON aa.Sku COLLATE SQL_Latin1_General_CP1_CI_AS = bb.[EVP SKU] COLLATE SQL_Latin1_General_CP1_CI_AS
and aa.UomCode COLLATE SQL_Latin1_General_CP1_CI_AS= bb.[EVP UOM CODE] COLLATE SQL_Latin1_General_CP1_CI_AS
),

Ventas as (
    SELECT * 
    FROM [DWH-MaxWarehouse].[Fact].[Sales]
    WHERE SalesOrderDateTime >= DATEADD(MONTH, -4, CAST(GETDATE() AS DATETIME))
      AND SalesOrderDateTime < DATEADD(DAY, 1, CAST(GETDATE() AS DATETIME))
)

select distinct * from EJD_DIMS 

"""

q1 = pd.read_sql(query, con=engine)

In [7]:
df_merged = pd.merge(q1, df_ventas, left_on=['trackingnum', 'Sku'], right_on=['TrackingNumber', 'ParentSku'], how='left')

In [8]:
df = df_merged.copy()

In [9]:
df = df.rename(columns={
    "EJD Weight": "EJD_Weight",
    "EJD Height": "EJD_Height",
    "EJD Length": "EJD_Length",
    "EJD Width": "EJD_Width"
})

## Limpiar UOMCODE

In [10]:
def normalize_uom_code(df, column='UomCode'):
    """
    Normaliza la columna UomCode:
    - Convierte a mayúsculas
    - Elimina espacios innecesarios
    - Reemplaza valores según las reglas definidas
    - Convierte formatos como "CASE_X" → "CS/X" y "PACK OF X" → "PK/X"
    
    Parámetros:
        df (pd.DataFrame): DataFrame con la columna UomCode
        column (str): Nombre de la columna a normalizar
    
    Retorna:
        pd.DataFrame: DataFrame con una nueva columna 'UomCode_N' normalizada
    """
    df = df.copy()
    
    # Convertir a mayúsculas y eliminar espacios innecesarios
    df['UomCode_N'] = df[column].str.strip().str.upper()

    # Reemplazar "CASE_X" → "CS/X" antes de otros reemplazos
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^CASE[_\s]?(\d+)', r'CS/\1', regex=True)

    # Reemplazar caracteres no uniformes y homogenizar
    replacements = {
        r'\bEACH\b': 'EA',  # EACH → EA
        r'\bEA\b': 'EA',    # Variaciones de EA → EA
        r'\bCASE\b': 'CS',  # CASE → CS
        r'\bPACK\b': 'PK',  # PACK → PK
        r'_' : '/'         # Reemplazar "_" por "/"
    }
    
    df['UomCode_N'] = df['UomCode_N'].replace(replacements, regex=True)

    # Convertir "PACK/X" a "PK/X"
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^PACK/', 'PK/', regex=True)
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^PA/', 'PK/', regex=True)

    # Convertir "CS OF X" → "CS_X" y "PK OF X" → "PK_X"
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^CS OF (\d+)', r'CS/\1', regex=True)
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^PK OF (\d+)', r'PK/\1', regex=True)

    # Reemplazar '/' por '_'
    df['UomCode_N'] = df['UomCode_N'].str.replace('/', '_', regex=False)

    # Reemplazar "ACE" por "EA" y quitar el resto de la cadena
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^ACE.*', 'EA', regex=True)

    return df

In [11]:
df = normalize_uom_code(df)

## Tomar el valor más reciente para pesos y dimensiones por transaccion

In [12]:
# Asegurar que las fechas son tipo datetime
df['TranDate'] = pd.to_datetime(df['TranDate'])
df['InvDate'] = pd.to_datetime(df['InvDate'])

# Ordenar primero 
df = df.sort_values(['TranDate', 'trackingnum', 'Sku', 'UomCode_N', 'InvDate'], ascending=[True, True, True, True, False])

# Obtener el índice del invdate más reciente dentro de cada grupo
idx = df.groupby(['TranDate', 'trackingnum', 'Sku', 'UomCode_N'])['InvDate'].idxmax()

# Seleccionar las filas correspondientes
df_mas_reciente = df.loc[idx].reset_index(drop=True)

In [13]:
# Asegurar que las fechas son tipo datetime

# Ordenar primero 
df_mas_reciente2 = df_mas_reciente.sort_values(['TranDate', 'trackingnum', 'Sku', 'UomCode_N', 'InvDate'], ascending=[False, True, True, True, True])

# Obtener el índice del invdate más reciente dentro de cada grupo
idx = df_mas_reciente2.groupby(['InvDate', 'trackingnum', 'Sku', 'UomCode_N'])['TranDate'].idxmax()

# Seleccionar las filas correspondientes
df_mas_reciente2 = df_mas_reciente2.loc[idx].reset_index(drop=True)

In [14]:
# Eliminar trackings repetidos por fechas

# Ordenar y seleccionar la fila con invdate más reciente para cada grupo de ['trackingnum', 'Sku', 'UomCode_N']
idx = df_mas_reciente2.groupby(['trackingnum', 'Sku', 'UomCode_N'])['InvDate'].idxmax()

# 3. Filtrar las filas
df_mas_reciente3 = df_mas_reciente2.loc[idx].reset_index(drop=True)

df = df_mas_reciente3

In [15]:
df2 = df.sort_values(by=['TranDate', 'trackingnum', 'Sku', 'UomCode_N', 'InvDate'], ascending=[True, True, True, True, False])

cols_to_fill = ['Weight', 'Height', 'Length', 'Width']
df2[cols_to_fill] = df2[cols_to_fill].replace(0, pd.NA)

df2[cols_to_fill] = df2.groupby(['TranDate', 'trackingnum', 'Sku', 'UomCode_N'])[cols_to_fill].transform(lambda x: x.bfill())

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\179128486.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2[cols_to_fill] = df2.groupby(['TranDate', 'trackingnum', 'Sku', 'UomCode_N'])[cols_to_fill].transform(lambda x: x.bfill())
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\179128486.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2[cols_to_fill] = df2.groupby(['TranDate', 'trackingnum', 'Sku', 'UomCode_N'])[cols_to_fill].transform(lambda x: x.bfill())


## Limpieza

In [16]:
# Ordenar por TranDate, trackingnum, Sku, UomCode_N, y InvDate DESC
df2 = df2.sort_values(by=['TranDate', 'trackingnum', 'Sku', 'UomCode_N', 'InvDate'], ascending=[True, True, True, True, False])

# Asignar ranking dentro de cada grupo según InvDate DESC (el primero será Rank 1)
df2['Rank'] = df2.groupby(['TranDate', 'trackingnum', 'Sku', 'UomCode_N']).cumcount() + 1

# Filtrar solo los de Ranking = 1
df_final = df2[df2['Rank'] == 1]

In [17]:
# Weight con nulos relleno con la moda

# Función para obtener la moda, con manejo de múltiples modas
def obtener_moda(grupo):
    moda = grupo.mode()
    if not moda.empty:
        return moda.iloc[0]  # toma la primera si hay varias
    else:
        return np.nan

# Calcular la moda por combinación de Sku y UomCode_N
moda_por_sku_uom = (
    df_final.groupby(['Sku', 'UomCode_N'])['Weight']
    .transform(lambda x: x.fillna(obtener_moda(x)))
)

# Reemplazar los valores nulos en Weight con la moda correspondiente
df_final['Weight'] = df_final['Weight'].fillna(moda_por_sku_uom)

In [18]:
# Eliminar nulos

df_final = df_final.dropna(subset=['TranDate', 'InvDate', 'trackingnum', 'Sku', 'UomCode_N', 'Weight', 'Weight', 'Length', 'Width'])
df_final.fillna(0, inplace=True)

In [19]:
#Convertir tipo de dato

df_final['Weight'] = df_final['Weight'].astype(str).astype(float)
df_final['Height'] = df_final['Height'].astype(str).astype(float)
df_final['Length'] = df_final['Length'].astype(str).astype(float)
df_final['Width'] = df_final['Width'].astype(str).astype(float)
df_final['EJD_Weight'] = df_final['EJD_Weight'].astype(str).astype(float)
df_final['EJD_Length'] = df_final['EJD_Length'].astype(str).astype(float)
df_final['EJD_Height'] = df_final['EJD_Height'].astype(str).astype(float)
df_final['EJD_Width'] = df_final['EJD_Width'].astype(str).astype(float)
df_final['TranDate'] = pd.to_datetime(df_final['TranDate'], format='%d-%m-%Y')
df_final['InvDate'] = pd.to_datetime(df_final['InvDate'], format='%d-%m-%Y')
df_final['UomQuantity'] = df_final['UomQuantity'].astype(str).astype(float).round(1)

In [20]:
# Ordenar Dimensiones de mayor a menor
df_sorted = df_final.copy()
df_sorted[['Height', 'Length', 'Width']] = df_final[['Height', 'Length', 'Width']].apply(
    lambda row: sorted(row, reverse=True), axis=1, result_type='expand')

df_sorted[['EJD_Height', 'EJD_Length', 'EJD_Width']] = df_final[['EJD_Height', 'EJD_Length', 'EJD_Width']].apply(
    lambda row: sorted(row, reverse=True), axis=1, result_type='expand')

In [21]:
df_sorted2 = df_sorted[['trackingnum', 'Sku', 'UomCode', 'Weight', 'Height', 'Length', 'Width']]

In [22]:
df_sorted2.rename(columns={
    'Weight': 'Weight_Original',
    'Height': 'Height_Original',
    'Length': 'Length_Original',
    'Width': 'Width_Original'
}, inplace=True)

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\1575963599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sorted2.rename(columns={


## Outliers

In [23]:
# Función para reemplazar outliers por NaN por Sku usando IQR
def replace_outliers_with_nan(group):
    Q1 = group[['Weight', 'Height', 'Length', 'Width']].quantile(0.25)
    Q3 = group[['Weight', 'Height', 'Length', 'Width']].quantile(0.75)
    IQR = Q3 - Q1
    factor = 1.5

    # Detectar outliers
    is_outlier = ((group[['Weight', 'Height', 'Length', 'Width']] < (Q1 - factor * IQR)) |
                  (group[['Weight', 'Height', 'Length', 'Width']] > (Q3 + factor * IQR)))

    # Reemplazar outliers por NaN
    group[['Weight', 'Height', 'Length', 'Width']] = group[['Weight', 'Height', 'Length', 'Width']].mask(is_outlier, np.nan)

    return group

# Aplicar la función por cada Sku
df_cleaned = df_sorted.groupby(['Sku', 'UomCode_N']).apply(replace_outliers_with_nan)

# Reiniciar el índice si deseas (opcional)
df_cleaned = df_cleaned.reset_index(drop=True)

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\3628699790.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_cleaned = df_sorted.groupby(['Sku', 'UomCode_N']).apply(replace_outliers_with_nan)


In [24]:
# Rellenar Vacios de outliers con la mediana
df_cleaned[['Weight', 'Height', 'Length', 'Width']] = df_cleaned.groupby(['Sku', 'UomCode_N'])[['Weight', 'Height', 'Length', 'Width']].transform(lambda x: x.fillna(x.median()))

## MODELO PREDICTIVO

### Agrupar por tipo de caja (semejanza entre dimensiones de la caja)

In [25]:
df = df_cleaned.copy()

In [26]:
# Asegurar que dimensiones son numéricas
df[['Height', 'Length', 'Width']] = df[['Height', 'Length', 'Width']].astype(float)

# Definir tolerancia
tolerancia = 2

# Agrupar por SKU y UomCode y aplicar "agrupación por bins"
df['dim_group'] = (
    df.groupby(['Sku', 'UomCode'], group_keys=False)
      .apply(lambda g: (
          ((g[['Height', 'Length', 'Width']] // tolerancia)
           .astype(int)
           .astype(str)
           .agg('_'.join, axis=1))
      ))
)

# Ahora convertir esas combinaciones a códigos únicos (grupos numéricos)
df['grupo_dim'] = df.groupby(['Sku', 'UomCode', 'dim_group'], group_keys=False).ngroup()

# Opcional: eliminar columna temporal
df.drop(columns='dim_group', inplace=True)

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\2918129312.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (


### Obtener el tipo de caja mas frecuente para las dims

In [27]:
# Agrupamos y tomamos el máximo por Sku, UomCode y grupo_dim
dimensiones_max = df.groupby(['Sku', 'UomCode', 'grupo_dim']).agg({
    'Height': 'max',
    'Length': 'max',
    'Width': 'max',
    'Weight': 'mean',
    'trackingnum': 'count'  # para saber cuántas veces aparece ese grupo
}).rename(columns={'trackingnum': 'n'})

# Agregar columna con criterio de desempate
dimensiones_max['criterio'] = (
    dimensiones_max['Height'] + 2 * dimensiones_max['Length'] + 2 * dimensiones_max['Width']
)

# Ordenar por frecuencia y criterio
dimensiones_final = (
    dimensiones_max
    .reset_index()
    .sort_values(['Sku', 'UomCode', 'n', 'criterio'], ascending=[True, True, False, False])
    .groupby(['Sku', 'UomCode'])
    .first()
    .reset_index()
)

# Redondear dimensiones a 0 decimales
dimensiones_final[['Height', 'Length', 'Width']] = (
    dimensiones_final[['Height', 'Length', 'Width']].round(0)
)

# Redondear peso a 2 decimales
dimensiones_final['Weight'] = (dimensiones_final[ 'Weight'].round(2))

## Armado Bases Finales

### Base Dims EJD

In [28]:
df_supplier = df_cleaned[["Sku", "UomCode", "UomQuantity", "BaseUnit", "EJD_Weight", "EJD_Height", "EJD_Length", "EJD_Width"]].drop_duplicates()
df_supplier = df_supplier.groupby(["Sku", "UomCode", "UomQuantity", "BaseUnit"])[["EJD_Weight", "EJD_Height", "EJD_Length", "EJD_Width"]].max().reset_index()
df_supplier["BaseUnit"] = df_supplier["BaseUnit"].astype(str).str.lower().map({"true": True, "false": False})
df_supplier = df_supplier.sort_values(by="BaseUnit", ascending=False).drop_duplicates(subset=["Sku", "UomCode"])

### Limpieza columna BaseUnit

In [29]:
# Si UomQty es 1 se toma como BaseUnit ese sku

# Base predicciones
index_bu_null = df_supplier[df_supplier["BaseUnit"].isna()].index
df_supplier.loc[index_bu_null, 'BaseUnit'] = True

# Base historico
index_bu_null = df_cleaned[df_cleaned["BaseUnit"].isna()].index
df_cleaned.loc[index_bu_null, 'BaseUnit'] = True

df_supplier.loc[df_supplier[df_supplier['BaseUnit'] == True].index, 'BaseUnit'] = True

In [30]:
# Limpieza base cleaned
true_clean_i = df_cleaned[df_cleaned['BaseUnit'] == 'True'].index
df_cleaned.loc[true_clean_i, 'BaseUnit'] = True

false_clean_i = df_cleaned[df_cleaned['BaseUnit'] == 'False'].index
df_cleaned.loc[false_clean_i, 'BaseUnit'] = False

In [31]:
df_supplier_na_base = df_supplier[df_supplier["BaseUnit"].isna()]
df_cleaned_na_base = df_cleaned[df_cleaned["BaseUnit"] == 0]

### Union con base Final

In [32]:
final_w_supplier = pd.merge(dimensiones_final, df_supplier, on=["Sku", "UomCode"], how="left")

In [33]:
df_pred = final_w_supplier[final_w_supplier['BaseUnit'] == True]

In [34]:
df_pred = df_pred[["Sku", "UomCode", "UomQuantity", 'Weight', 'Height', 'Length', 'Width']]

### Aplicar Threshold a Predicciones (solo como alerta de control)

Aplicar a Skus que son unidades base unicamente

In [35]:
final_w_supplierT = final_w_supplier[final_w_supplier['BaseUnit'] == True]
final_w_supplierF = final_w_supplier[final_w_supplier['BaseUnit'] == False]

df_cleanedT = df_cleaned[df_cleaned['BaseUnit'] == True]
df_cleanedF = df_cleaned[df_cleaned['BaseUnit'] == False]

In [36]:
#### PREDICCIONES
# Threshold para skus con peso menor a 1
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_weight"] = final_w_supplierT["EJD_Weight"] + 0.31
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_height"] = final_w_supplierT["EJD_Height"] + 3
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_length"] = final_w_supplierT["EJD_Length"] + 3
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_width"] = final_w_supplierT["EJD_Width"] + 3

# Threshold para skus con peso mayor a 1
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] >= 1, "th_weight"] = final_w_supplierT["EJD_Weight"] + 3
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] >= 1, "th_height"] = final_w_supplierT["EJD_Height"] + 5
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] >= 1, "th_length"] = final_w_supplierT["EJD_Length"] + 5
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] >= 1, "th_width"] = final_w_supplierT["EJD_Width"] + 5

#### HISTORICO
# Threshold para skus con peso menor a 1
df_cleanedT.loc[df_cleanedT["EJD_Weight"] < 1, "th_weight"] = df_cleanedT["EJD_Weight"] + 0.31
df_cleanedT.loc[df_cleanedT["EJD_Weight"] < 1, "th_height"] = df_cleanedT["EJD_Height"] + 3
df_cleanedT.loc[df_cleanedT["EJD_Weight"] < 1, "th_length"] = df_cleanedT["EJD_Length"] + 3
df_cleanedT.loc[df_cleanedT["EJD_Weight"] < 1, "th_width"] = df_cleanedT["EJD_Width"] + 3

# Threshold para skus con peso mayor a 1
df_cleanedT.loc[df_cleanedT["EJD_Weight"] >= 1, "th_weight"] = df_cleanedT["EJD_Weight"] + 3
df_cleanedT.loc[df_cleanedT["EJD_Weight"] >= 1, "th_height"] = df_cleanedT["EJD_Height"] + 5
df_cleanedT.loc[df_cleanedT["EJD_Weight"] >= 1, "th_length"] = df_cleanedT["EJD_Length"] + 5
df_cleanedT.loc[df_cleanedT["EJD_Weight"] >= 1, "th_width"] = df_cleanedT["EJD_Width"] + 5

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\778330037.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_weight"] = final_w_supplierT["EJD_Weight"] + 0.31
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\778330037.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_height"] = final_w_supplierT["EJD_Height"] + 3
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\778

In [37]:
# Pesos y dimensiones finales (aplicando el threshold)

#### Sobrepasando limite superior del rango ########
final_w_supplierT.loc[final_w_supplierT["Weight"] > final_w_supplierT["th_weight"], "fail_weight"] = 1
final_w_supplierT["Final_Weight"] = final_w_supplierT["Weight"]

final_w_supplierT.loc[final_w_supplierT["Height"] > final_w_supplierT["th_height"], "fail_eight"] = 1
final_w_supplierT["Final_Height"] = final_w_supplierT["Height"]

final_w_supplierT.loc[final_w_supplierT["Length"] > final_w_supplierT["th_length"], "fail_length"] = 1
final_w_supplierT["Final_Length"] = final_w_supplierT["Length"]

final_w_supplierT.loc[final_w_supplierT["Width"] > final_w_supplierT["th_width"], "fail_width"] = 1
final_w_supplierT["Final_Width"] = final_w_supplierT["Width"]


#### Sobrepasando limite inferior del rango ########
final_w_supplierT.loc[final_w_supplierT["Weight"] < final_w_supplierT["EJD_Weight"], "fail_weight"] = 1
final_w_supplierT.loc[final_w_supplierT["Height"] < final_w_supplierT["EJD_Height"], "fail_eight"] = 1
final_w_supplierT.loc[final_w_supplierT["Length"] < final_w_supplierT["EJD_Length"], "fail_length"] = 1
final_w_supplierT.loc[final_w_supplierT["Width"] < final_w_supplierT["EJD_Width"], "fail_width"] = 1

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\3936553958.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierT.loc[final_w_supplierT["Weight"] > final_w_supplierT["th_weight"], "fail_weight"] = 1
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\3936553958.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierT["Final_Weight"] = final_w_supplierT["Weight"]
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\3936553958.py:7: SettingWithCopyWarning: 
A value is tryi

In [38]:
# Pesos y dimensiones finales (aplicando el threshold)

#### Skus que no tienen dimensiones en EJD ########

final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "fail_weight"] = np.nan
final_w_supplierF["Final_Weight"] = final_w_supplierF["Weight"]

final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "fail_eight"] = np.nan
final_w_supplierF["Final_Height"] = final_w_supplierF["Height"]

final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "fail_length"] = np.nan
final_w_supplierF["Final_Length"] = final_w_supplierF["Length"]

final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "fail_width"] = np.nan
final_w_supplierF["Final_Width"] = final_w_supplierF["Width"]

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\2575437867.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierF["Final_Weight"] = final_w_supplierF["Weight"]
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\2575437867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierF["Final_Height"] = final_w_supplierF["Height"]
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\2575437867.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [39]:
# Quitando dimensiones a Skus que no son unidad base

final_w_supplierF[['EJD_Weight', 'EJD_Height', 'EJD_Length', 'EJD_Width']] = np.nan

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\1779026084.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierF[['EJD_Weight', 'EJD_Height', 'EJD_Length', 'EJD_Width']] = np.nan


In [40]:
final_w_supplier = pd.concat([final_w_supplierT, final_w_supplierF], ignore_index=True)

### Metricas de Desviacion con la Especificacion

In [41]:
# SUMAR DIMENSIONES
df_cleanedT["Sum_Dim"] = df_cleanedT["Height"] + df_cleanedT["Length"] + df_cleanedT["Width"]
df_cleanedF["Sum_Dim"] = df_cleanedF["Height"] + df_cleanedF["Length"] + df_cleanedF["Width"]
df_cleanedT["Sum_Dim_EJD_Sup"] = df_cleanedT["th_height"] + df_cleanedT["th_length"] + df_cleanedT["th_width"]
df_cleanedT["Sum_Dim_EJD_Low"] = df_cleanedT["EJD_Height"] + df_cleanedT["EJD_Length"] + df_cleanedT["EJD_Width"]

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\172691531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleanedT["Sum_Dim"] = df_cleanedT["Height"] + df_cleanedT["Length"] + df_cleanedT["Width"]
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\172691531.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleanedF["Sum_Dim"] = df_cleanedF["Height"] + df_cleanedF["Length"] + df_cleanedF["Width"]
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\172691531.py:4: SettingWithCopyWarning

In [42]:
# MEDIANA PESO Y DIMS
df_metricaT = df_cleanedT.groupby(["Sku", 'UomCode_N', 'BaseUnit'])[["Weight", "Sum_Dim", "EJD_Weight", "th_weight", "Sum_Dim_EJD_Low", "Sum_Dim_EJD_Sup"]].median().round(2).reset_index()
df_metricaF = df_cleanedF.groupby(["Sku", 'UomCode_N', 'BaseUnit'])[['Weight', "Sum_Dim"]].median().round(2).reset_index()

In [43]:
# FLAG PESO
i_w_low = df_metricaT[(df_metricaT["Weight"] < df_metricaT["EJD_Weight"])].index
i_w_high = df_metricaT[(df_metricaT["Weight"] > df_metricaT["th_weight"])].index

df_metricaT["Weight_Flag"] = "Within range"
df_metricaT.loc[i_w_low, "Weight_Flag"] = "Underweight"
df_metricaT.loc[i_w_high, "Weight_Flag"] = "Overweight"

df_metricaF["Weight_Flag"] = "No EJD Dims"

In [44]:
# FLAG DIMS
i_w_lowD = df_metricaT[(df_metricaT["Sum_Dim"] < df_metricaT["Sum_Dim_EJD_Low"])].index
i_w_highD = df_metricaT[(df_metricaT["Sum_Dim"] > df_metricaT["Sum_Dim_EJD_Sup"])].index

df_metricaT["DIM_Flag"] = "Within range"
df_metricaT.loc[i_w_lowD, "DIM_Flag"] = "Under"
df_metricaT.loc[i_w_highD, "DIM_Flag"] = "Over"

df_metricaF["DIM_Flag"] = "No EJD Dims"

In [45]:
df_cleaned = pd.concat([df_cleanedT, df_cleanedF], ignore_index=True)
df_metrica = pd.concat([df_metricaT, df_metricaF], ignore_index=True)

# SALIDA EVP PARA SUBIR POR MEDIO DEL UOMCODE

In [46]:
df_pred_evp = df_pred[['Sku', 'UomCode', 'UomQuantity', 'Weight', 'Height', 'Length', 'Width']]

In [47]:
from datetime import datetime
fecha = datetime.today().strftime('%Y%m%d')

# Suponiendo que tu DataFrame se llama df
chunk_size = 5000  # Tamaño de cada archivo
chunks = np.array_split(df_pred_evp, len(df_pred_evp) // chunk_size + (1 if len(df_pred_evp) % chunk_size != 0 else 0))  # Dividir en partes

# Ruta donde quieres guardar
ruta_base = r'C:\Users\ecalderon\OneDrive - BITS - Max Warehouse\Shipping\Weights Process'

# Guardar cada chunk en un CSV dentro de la ruta especificada
list(map(lambda x: x[1].to_csv(os.path.join(ruta_base, f"{fecha}_df_evp_{x[0]+1}.csv"), index=False), enumerate(chunks)))

print(f"Se han creado {len(chunks)} archivos CSV.")

Se han creado 4 archivos CSV.


C:\Users\ecalderon\AppData\Local\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


# TABLA HISTÓRICA PREDICCIONES PARA SQL

In [48]:
from datetime import date

df_pred_evp['Fecha'] = date.today()

In [49]:
# Datos de conexión
servidor = '10.12.200.59'
base_de_datos = 'Maxwarehouse'

# Crear la conexión con autenticación de Windows
engine = create_engine(f"mssql+pyodbc://@{servidor}/{base_de_datos}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Nombre de la tabla destino
table_name = 'DataPrediccionDims'

try:
    # Insertar datos en la tabla, sin duplicar registros, asegurando que se mantenga el índice único
    df_pred_evp.to_sql(table_name, con=engine, if_exists='append', index=False)
    print("✅ Registros insertados correctamente.")
except Exception as e:
    print(f"⚠️ Error al insertar los registros: {e}")

⚠️ Error al insertar los registros: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.DataPrediccionDims' with unique index 'idx_unique_sku_uom_fecha'. The duplicate key value is (ACE-1001262, EA, 2025-05-06). (2601) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")
[SQL: INSERT INTO [DataPrediccionDims] ([Sku], [UomCode], [UomQuantity], [Weight], [Height], [Length], [Width], [Fecha]) VALUES (?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?) ... 6582 characters truncated ... , ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ('ACE-1001262', 'EA', 1.0, 2.9, 10.0, 8.0, 8.0, datetime.datetime(2025, 5, 6, 0, 0), 'ACE-1001296', 'EA', 1.0, 2.95, 10.0, 8.0, 8.0, datetime.datetime(2025, 5,

# Ventas

In [50]:
df_ventas = df_cleaned[['TranDate', 'InvDate', 'trackingnum','Sku', 'UomCode_N', 'UomQuantity', 'SalesStatusEVP','TotalSales', 'TotalCost']].drop_duplicates()

In [51]:
# crear campos de temporalidad

df_ventas["Year"] = df_ventas["TranDate"].dt.year
df_ventas["Month"] = df_ventas["TranDate"].dt.month_name()
df_ventas["YearMonth"] = df_ventas["TranDate"].dt.strftime("%Y-%m")

In [52]:
# Agrupar por Año-Mes y SKU, sumando las ventas
ventas_por_mes = df_ventas.groupby(["YearMonth", "Sku", "UomCode_N"])["TotalSales"].sum().reset_index()
ventas_por_mes = ventas_por_mes[ventas_por_mes['TotalSales'] != 0.00]

In [53]:
# Colocar los meses en columnas

df_transposed = ventas_por_mes.pivot_table(index=["Sku", 'UomCode_N'], columns="YearMonth", values="TotalSales", aggfunc="sum").round(2).reset_index()

In [54]:
# Calcular tendencia

# Columnas de meses
ventas_cols = df_transposed.columns[2:]

# Calcular la variación promedio mes a mes
df_transposed["Tendencia"] = df_transposed[ventas_cols].pct_change(axis=1).mean(axis=1, skipna=True).round(2)

# Reemplazar NaN por 0 si no hubo ventas en ningún mes
df_transposed = df_transposed.fillna(0)

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\3470377078.py:7: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_transposed["Tendencia"] = df_transposed[ventas_cols].pct_change(axis=1).mean(axis=1, skipna=True).round(2)


In [55]:
df_metrica.isnull().sum()

Sku                   0
UomCode_N             0
BaseUnit              0
Weight                0
Sum_Dim               0
EJD_Weight         3921
th_weight          3921
Sum_Dim_EJD_Low    3921
Sum_Dim_EJD_Sup    3921
Weight_Flag           0
DIM_Flag              0
dtype: int64

### Unir al dataframe de Metricas

In [56]:
df_metrica = df_metrica.merge(df_transposed, on=["Sku", "UomCode_N"], how="left")

## CSVs

In [57]:
ruta_base = r'C:\Users\ecalderon\OneDrive - BITS - Max Warehouse\Shipping\Weights Process'

# Guardar los otros DataFrames en la misma ruta
final_w_supplier.to_csv(os.path.join(ruta_base, "df_predicciones.csv"), index=False)
df_sorted.to_csv(os.path.join(ruta_base, "df_historico.csv"), index=False)
df_metrica.to_csv(os.path.join(ruta_base, "df_metrica.csv"), index=False)


## Query de Salida

### Por tracking

In [58]:
server = '10.12.200.59' #ip del DB es 10.11.100.103
database = 'Maxwarehouse'

# Crear el engine usando la cadena de conexión para SQL Server con pyodbc
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Ejecutar consulta 

query2 = """

WITH 

endicia_semana_mes AS (
    SELECT DISTINCT  
        [Postmark_Date] AS TranDate, 
        [Transaction_Date (GMT)] AS InvDate, 
        REPLACE([Tracking Number], '''', '') AS trackingnum, 
		[Transaction_Amount] as DiscountedNetCharge
    FROM Endiciatransacprovision
    WHERE ISDATE([Transaction_Date (GMT)]) = 1
      AND CAST([Transaction_Date (GMT)] AS date) >= DATEADD(MONTH, -4, GETDATE())
    
    UNION ALL
    
    SELECT DISTINCT  
        [Postmark Date] AS TranDate, 
        [Transaction Date (GMT)] AS InvDate, 
        REPLACE([Tracking Number], '''', '') AS trackingnum, 
		[Transaction Amount] as DiscountedNetCharge
    FROM endicia_semanal
    WHERE ISDATE([Transaction Date (GMT)]) = 1
      AND CAST([Transaction Date (GMT)] AS date) >= DATEADD(MONTH, -2, GETDATE())
),

Factura3meses AS (

    -- FEDEX
    SELECT DISTINCT
        CAST(InvoiceDate AS date) AS TranDate,
        CAST(InvoiceDate AS date) AS InvDate,
        GroundTrackingIDPrefix + ExpressorGroundTrackingID AS trackingnum,
        max(NetChargeAmount) as DiscountedNetCharge
    FROM 
        FEDEXHISTORICO
    GROUP BY 
        CAST(InvoiceDate AS date),
        CAST(InvoiceDate AS date),
        GroundTrackingIDPrefix + ExpressorGroundTrackingID

    UNION ALL
    -- UPS (CON PRIORIDAD PackageDimensions)
    SELECT 
    cast(TransactionDate as date), cast(InvoiceDate as date), TrackingNumber, sum(netamount) as DiscountedNetCharge
	FROM UPSNEWFORMAT
	GROUP BY 
	cast(TransactionDate as date), cast(InvoiceDate as date), TrackingNumber

    UNION ALL
    -- USPS (Endicia)
    SELECT DISTINCT 
        TranDate,
        InvDate,
        trackingnum,
        DiscountedNetCharge
    FROM endicia_semana_mes
),

DWVentasUnicas as (
	Select 
		SalesOrderNumber, Sku
	from DWShippingInfo as dws
	where cast(SalesOrderDate as date ) between dateadd(MONTH,-4, cast(GETDATE() as date)) and cast(GETDATE() as date)
	group by SalesOrderNumber, Sku
),

CantidadSkusxVenta as (
	select
		SalesOrderNumber, count(Sku) CantidadSkus
	from DWVentasUnicas dw 
	group by SalesOrderNumber
	having count(Sku) =1
),

PesosfacturaVentas as (
	select  
		TranDate, InvDate, dw.trackingnum, mw.FulfillmentOrderNumber, dw.DiscountedNetCharge, 
		mw.Sku, mw.UomCode, mw.UomQuantity, mw.EstimatedShippingCost
	FROM Factura3meses dw
	INNER JOIN DWShippingInfo  mw
		ON dw.trackingnum = mw.trackingnum
	WHERE dw.trackingnum !=''
	and SalesOrderNumber in (select SalesOrderNumber from CantidadSkusxVenta)
	and cast(UomQuantity as float)/cast(Quantity as float)= 1
)

select InvDate,trackingnum, FulfillmentOrderNumber, sum(DiscountedNetCharge) as DiscountedNetCharge, Sku, UomCode, UomQuantity, 
max(EstimatedShippingCost) as EstimatedCost  
from PesosfacturaVentas
group by InvDate,trackingnum, FulfillmentOrderNumber, Sku, UomCode, UomQuantity
having InvDate >= DATEADD(WEEK, -2, GETDATE())

"""

campos_salida = pd.read_sql(query2, con=engine)

In [59]:
campos_salida = normalize_uom_code(campos_salida)

In [60]:
base_salida = final_w_supplier[["Sku", "UomCode", "Final_Height", "Final_Length", "Final_Width", "Final_Weight"]]

In [61]:
salida_evp = pd.merge(campos_salida, base_salida, on=["Sku", "UomCode"], how="left")

In [62]:
salida_evp = salida_evp[['trackingnum', 'FulfillmentOrderNumber', "Final_Height", "Final_Length", "Final_Width", "Final_Weight", 'DiscountedNetCharge']]
salida_evp.rename(columns={"trackingnum": "TrackingNumber", 
                           "FulfillmentOrderNumber": "ShipperReference",
                           "Final_Height": "PackageHeight",
                           "Final_Length": "PackageLength",
                           "Final_Width": "PackageWidth",
                           "Final_Weight": "BilledWeight"}, inplace=True)

In [63]:
#salida_evp.to_csv("df_evp.csv", index=False)

## Comparacion estimated vs real cost

In [64]:
campos_salida.rename(columns={'DiscountedNetCharge': 'Cost'}, inplace=True)

In [65]:
# convertir tipo de dato
campos_salida['Cost'] = pd.to_numeric(campos_salida['Cost'], errors='coerce')
campos_salida['EstimatedCost'] = pd.to_numeric(campos_salida['EstimatedCost'], errors='coerce')

In [66]:
# Variacion porcentual del costo
campos_salida['variacion_costo'] = ((campos_salida['Cost'] - campos_salida['EstimatedCost']) / campos_salida['EstimatedCost']).round(2)

In [67]:
# Semana del reporte
campos_salida['Semana'] = campos_salida['InvDate'].dt.isocalendar().week

In [68]:
campos_salida[(campos_salida['variacion_costo'] < 0) & (campos_salida['EstimatedCost'] != 0)].sort_values('variacion_costo', ascending=True)

,InvDate,trackingnum,FulfillmentOrderNumber,Cost,Sku,UomCode,UomQuantity,EstimatedCost,UomCode_N,variacion_costo,Semana
38109,2025-05-03 00:00:00.000,1ZC6583AYW50316837,PO-259261721-EJD,-15.23,EJD-2015165,EA,1.00000,7.00,EA,-3.18,18
26305,2025-05-03 00:00:00.000,1Z53E7A90300923856,PO-259290138-EJD,-64.38,EJD-8093917,EA,1.00000,33.49,EA,-2.92,18
34807,2025-05-03 00:00:00.000,1Z53FA660350545814,PO-259292573-EJD,-15.71,EJD-7817018,EA,1.00000,9.37,EA,-2.68,18
13907,2025-04-26 00:00:00.000,1Z53FA660300539451,PO-259253057-EJD,-15.73,JEN-9001-0521,EA,1.00000,9.37,EA,-2.68,17
11888,2025-04-26 00:00:00.000,1Z53FA370300919729,PO-259262631-EJD,-15.73,JEN-1400-7439,EA,1.00000,9.79,EA,-2.61,17
...,...,...,...,...,...,...,...,...,...,...,...
34777,2025-05-03 00:00:00.000,1Z53FA660300558743,PO-259311688-EJD,9.42,ACE-41142,PACK_12,12.00000,9.49,PK_12,-0.01,18
29654,2025-05-03 00:00:00.000,1Z53E8A60350814026,PO-259315370-EJD,9.25,ACE-7284037,EA,1.00000,9.37,EA,-0.01,18
24017,2025-05-01 01:49:07.730,9400111899560755207201,PO-259326890-EJD,5.49,ACE-22680,EA,1.00000,5.53,EA,-0.01,18
32792,2025-05-03 00:00:00.000,1Z53E9290300426836,PO-259325982-EJD,9.25,ACE-7284037,EA,1.00000,9.37,EA,-0.01,18


In [84]:
# Ruta completa al archivo costos.csv
ruta_costos = os.path.join(ruta_base, 'costos.csv')

# Guardar en la ruta especificada (modo append)
campos_salida.to_csv(ruta_costos, mode='a', header=False, index=False)

## Cuantificación Impacto

In [70]:
server = '10.12.200.59' #ip del DB es 10.11.100.103
database = 'Maxwarehouse'

# Crear el engine usando la cadena de conexión para SQL Server con pyodbc
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Ejecutar consulta 

query = """
select * from [stg-MaxWarehouse].[EvpCsv].[EVPAlllSKUsDimNWeight]
"""

df_evp = pd.read_sql(query, con=engine)

### Skus que cambiaron sus dimensiones

In [71]:
df_evp = df_evp[['EVP SKU', 'EVP UOM CODE', 'EVP UOM QTY', 'EVP Weight', 'EVP Height', 'EVP Length', 'EVP Width']]

In [72]:
df_merge = df_pred.merge(
    df_evp,  
    left_on=['Sku', 'UomCode'], 
    right_on=['EVP SKU', 'EVP UOM CODE'], 
    how='left'
)

In [73]:
# EVP solo toma sku base unit
#df_bu = df_merge[df_merge['BaseUnit'] == True]

# Quitando nulos
df_bu = df_merge[~df_merge['EVP Width'].isnull()]

In [74]:
# convertir tipo de dato
df_bu['EVP Weight'] = pd.to_numeric(df_bu['EVP Weight'], errors='coerce')
df_bu['EVP Height'] = pd.to_numeric(df_bu['EVP Height'], errors='coerce')
df_bu['EVP Length'] = pd.to_numeric(df_bu['EVP Length'], errors='coerce')
df_bu['EVP Width'] = pd.to_numeric(df_bu['EVP Width'], errors='coerce')

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\3433237752.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bu['EVP Weight'] = pd.to_numeric(df_bu['EVP Weight'], errors='coerce')
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\3433237752.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bu['EVP Height'] = pd.to_numeric(df_bu['EVP Height'], errors='coerce')
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\3433237752.py:4: SettingWithCopyWarning: 
A value is trying to be set on a c

In [75]:
# Fecha del reporte
df_bu['Fecha'] = pd.to_datetime('today').normalize()
df_bu['Semana'] = df_bu['Fecha'].dt.isocalendar().week

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\1692906994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bu['Fecha'] = pd.to_datetime('today').normalize()
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\1692906994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bu['Semana'] = df_bu['Fecha'].dt.isocalendar().week


In [76]:
# Suma dimensiones
df_bu['Sum_Dims_Pred'] = df_bu['Height'] + df_bu['Length'] + df_bu['Width']
df_bu['Sum_Dims_EVP'] = df_bu['EVP Height'] + df_bu['EVP Length'] + df_bu['EVP Width']

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\436886150.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bu['Sum_Dims_Pred'] = df_bu['Height'] + df_bu['Length'] + df_bu['Width']
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\436886150.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bu['Sum_Dims_EVP'] = df_bu['EVP Height'] + df_bu['EVP Length'] + df_bu['EVP Width']


In [77]:
# Variacion porcentual
df_bu['variacion_weight'] = ((df_bu['Weight'] - df_bu['EVP Weight']) / df_bu['EVP Weight']).round(2)
df_bu['variacion_dims'] = ((df_bu['Sum_Dims_Pred'] - df_bu['Sum_Dims_EVP']) / df_bu['Sum_Dims_EVP']).round(2)

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\34305253.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bu['variacion_weight'] = ((df_bu['Weight'] - df_bu['EVP Weight']) / df_bu['EVP Weight']).round(2)
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_29804\34305253.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bu['variacion_dims'] = ((df_bu['Sum_Dims_Pred'] - df_bu['Sum_Dims_EVP']) / df_bu['Sum_Dims_EVP']).round(2)


In [78]:
df_bu = df_bu.rename(columns={'EVP Weight': 'EVP_Weight', 'EVP Height': 'EVP_Height', 'EVP Length': 'EVP_Length', 'EVP Width': 'EVP_Width'})

In [91]:
# Ruta completa al archivo
ruta_cuantificacion = os.path.join(ruta_base, 'cuantificacion_skus.csv')

# Guardar en la ruta especificada (modo append)
df_bu.to_csv(ruta_cuantificacion, mode='a', header=False, index=False)

In [94]:
df_bu

,Sku,UomCode,UomQuantity,Weight,Height,Length,Width,EVP SKU,EVP UOM CODE,EVP UOM QTY,EVP_Weight,EVP_Height,EVP_Length,EVP_Width,Fecha,Semana,Sum_Dims_Pred,Sum_Dims_EVP,variacion_weight,variacion_dims
0,ACE-1001262,EA,1.0,2.90,10.0,8.0,8.0,ACE-1001262,EA,1.00000,2.340,5.00,4.350,4.350,2025-05-06,19,26.0,13.700,0.24,0.90
1,ACE-1001296,EA,1.0,2.95,10.0,8.0,8.0,ACE-1001296,EA,1.00000,2.525,5.00,4.400,4.400,2025-05-06,19,26.0,13.800,0.17,0.88
2,ACE-1001320,EA,1.0,2.90,10.0,8.0,8.0,ACE-1001320,EA,1.00000,2.360,5.00,4.300,4.350,2025-05-06,19,26.0,13.650,0.23,0.90
3,ACE-10017,EA,1.0,4.68,10.0,8.0,8.0,ACE-10017,EA,1.00000,2.100,4.95,4.150,4.150,2025-05-06,19,26.0,13.250,1.23,0.96
4,ACE-1001874,EA,1.0,0.34,11.0,8.0,2.0,ACE-1001874,EA,1.00000,0.210,8.15,0.700,3.950,2025-05-06,19,21.0,12.800,0.62,0.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16027,STC-BTU25X20RWHH,EACH,1.0,11.00,33.0,24.0,3.0,STC-BTU25X20RWHH,EACH,1.00000,11.000,33.00,24.000,3.000,2025-05-06,19,60.0,60.000,0.00,0.00
16028,STC-BTU8X4SWHH,EACH,1.0,2.60,14.0,7.0,4.0,STC-BTU8X4SWHH,EACH,1.00000,2.600,14.00,7.000,4.000,2025-05-06,19,25.0,25.000,0.00,0.00
16029,TSM-MDN7020,EA,1.0,6.00,16.0,16.0,10.0,TSM-MDN7020,EA,1.00000,4.500,16.40,9.200,16.100,2025-05-06,19,42.0,41.700,0.33,0.01
16030,TSM-MDN7020-A,PACK_1,1.0,5.67,17.0,16.0,16.0,TSM-MDN7020-A,PACK_1,1.00000,23.000,17.00,16.000,16.000,2025-05-06,19,49.0,49.000,-0.75,0.00


In [80]:
def send_email_final():
    # Configuración del correo
    sender_email = "ecalderon@maxwarehouse.com"  
    sender_password = "Maxwarehouse2025$"  

    # 🔹 Lista de destinatarios
    recipient_list = ["ecalderon@maxwarehouse.com", "vpverbena@maxwarehouse.com"]
    
    # Contenido del correo
    subject = "🔔 Alerta: Generación de predicciones de pesos completada"
    body = "El modelo de predicción ha finalizado correctamente. Sube los resultados a EVP."

    # Crear el mensaje
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = ", ".join(recipient_list)

    try:
        # Conectar al servidor SMTP (ejemplo para Gmail)
        server = smtplib.SMTP("smtp.office365.com", 587)  # Usa "smtp.office365.com" para Outlook
        server.starttls()  # Seguridad TLS
        server.login(sender_email, sender_password)  # Autenticación
        server.sendmail(sender_email, recipient_list, msg.as_string())  # Enviar
        server.quit()
        print("✅ Correo enviado correctamente")
    except Exception as e:
        print(f"⚠ Error al enviar el correo: {e}")

# Llamar a la función al final del proceso
send_email_final()

✅ Correo enviado correctamente


In [81]:
import nbformat

# Cargar el notebook
with open("Scrip_Pesos_y_Dims_Modas_Limpio.ipynb", "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)

# Extraer el código de cada celda y guardarlo en un archivo .py
codigo = "\n".join(cell["source"] for cell in notebook["cells"] if cell["cell_type"] == "code")

# Guardar como .py
with open("Scrip_Pesos_y_Dims_Modas.py", "w", encoding="utf-8") as f:
    f.write(codigo)